In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [2]:
def calc_logloss(y, y_pred):
    try: 
        err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    except ValueError:
        
    return err

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.


In [3]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [160]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            #eta *= 1.1
            #print(i, W, err, eta)
            print(i, W, err)
        elif i + 1 == iterations:
            print(i, W, err, eta)
    return W

In [6]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [5]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [7]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [152]:
W = eval_model(X_st, y, iterations=10000, eta=3.21)

0 [-0.72111774 -4.79557623  0.55979739 -0.29429967] 1.1785958344356262 3.21
1000 [-24.64122519  -2.71018945  -6.4771752   20.41395284] 0.14482557957072034 3.21
2000 [-30.92250528  -3.17291396  -8.27998896  25.49741542] 0.1218174650599809 3.21
3000 [-35.62556183  -3.59546493  -9.60649947  29.29296965] 0.10977542501890966 3.21
4000 [-39.47539264  -3.9363358  -10.69306357  32.38462208] 0.10175473835704729 3.21
5000 [-42.79172009  -4.22444639 -11.63239508  35.03377036] 0.09583177523018097 3.21
6000 [-45.74517941  -4.47544996 -12.47321889  37.38002285] 0.0911545459909963 3.21
7000 [-48.43697047  -4.69874451 -13.24408687  39.50629322] 0.08728538695233008 3.21
8000 [-50.93171095  -4.90042762 -13.96302017  41.46567582] 0.08397508979322418 3.21
9000 [-53.27293533  -5.08471006 -14.64200158  43.29412899] 0.08107066581535485 3.21
9999 [-55.48900478  -5.25449286 -15.28866447  45.01540007] 0.0784751623725171 3.21


Ответ: лучший результат я получил при eta = 3.21 и iterations=10000

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [164]:
def calc_pred_proba(w, X):

    y_pred_proba = np.squeeze(sigmoid(np.dot(X, w)))

    return y_pred_proba

In [156]:
X_st.shape, W.shape

((10, 4), (4,))

In [165]:
Prob = calc_pred_proba(W, X_st)
Prob

array([3.20499012e-01, 8.60252532e-04, 1.00000000e+00, 1.50082183e-11,
       9.78155056e-01, 6.99247751e-03, 1.00000000e+00, 4.93871807e-03,
       6.95298072e-01, 1.00000000e+00])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [212]:
def calc_pred(w, X):

    y_pred = np.zeros(X.shape[0])
    
    A = calc_pred_proba(w, X)
    
    y_pred = (A > 0.5).astype(int)

    return y_pred

In [215]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

6. Могла ли модель переобучиться? Почему?

Думаю, что могла, как и любая модель. Потому что мы подгоняли веса так, чтобы минимизировать ошибку и не проверяли работу алгоритма.

5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [219]:
def accuracy(y, y_pred):

    TP = ((y == 1) & (y_pred == 1)).sum()
    FP = ((y == 0) & (y_pred == 1)).sum()
    FN = ((y == 1) & (y_pred == 0)).sum()
    TN = ((y == 0) & (y_pred == 0)).sum()
    
    conf_matrix =  np.array([[TP, FP], [FN, TN]])
    
    accuracy = (TP + TN) / (TP + FP + FN + TN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    
    return conf_matrix, np.array([accuracy, precision, recall])

In [221]:
accuracy(y,y_pred)

(array([[5, 0],
        [0, 5]]),
 array([1., 1., 1.]))

In [225]:
def Fb(y, y_pred, b=1):
    
    conf_matrix, metrics = accuracy(y, y_pred)
    
    Fb = ((1 + b**2) * metrics[1] * metrics[2])/(b**2 * (metrics[1] + metrics[2]))
    
    return Fb

In [226]:
Fb(y,y_pred)

1.0